# Phương trình đường cong
\begin{equation}
y^2 = x^3 + ax + b
\end{equation}

In [278]:
INF_POINT = None

class EllipticCurve:
	def __init__(self, a, b, p):
		self.a = a
		self.b = b
		self.p = p
		self.points = []
		# self.definePoints()

	#Hàm thêm tất cả các điểm thuộc được cong vào points
	# def definePoints(self):
	# 	self.points.append(INF_POINT)
	# 	for x in range(self.p):
	# 		for y in range(self.p):
	# 			if self.equalModp(y * y, x * x * x + self.a * x + self.b):
	# 				self.points.append((x,y))

	# Hàm tính toán phép cộng điểm trên đường cong elliptic
	def addition(self, P1, P2):
		if P1 == INF_POINT:
			return P2
		if P2 == INF_POINT:
			return P1

		x1, y1 = P1
		x2, y2 = P2

		if self.equalModp(x1, x2) and self.equalModp(y1, -y2): #Kiểm tra hai điểm có phải hai điểm đối xứng
			return INF_POINT

		if self.equalModp(x1, x2) and self.equalModp(y1, y2): # Kiểm tra hai điểm có giống nhau không
			s = self.reduceModp((3 * x1 * x1 + self.a) * pow(2 * y1, -1, self.p)) # Công thức tính s nếu hai điểm giống nhau
		else:
			s = self.reduceModp((y1 - y2) * pow(x1 - x2, -1, self.p)) # Công thức tính s nếu hai điểm khác nhau

		#Tính giá trị tạo độ điểm mới theo công thức
		x3 = self.reduceModp(s * s - x1 - x2)
		y3 = self.reduceModp(s * (x1 - x3) - y1)

		return (x3, y3)

	# Hàm tính toán phép nhân điểm với số nguyên trên đường cong elliptic áp dụng phương pháp nhân đôi và cộng
	def multiplication(self, P, n):
		if P == INF_POINT:
			return P

		current_point = None
		while n:
			if n & 1: # Thực hiện phép AND để kiểm tra bit cuối cùng của n có bằng 1 hay không
				current_point = self.addition(current_point, P)
			P = self.addition(P, P)
			n >>= 1  # Phép bitwise right shift
		return current_point


	def testAssociativity(self):
		n = len(self.points)
		for i in range (n):
			for j in range(n):
				for k in range(n):
					P = self.addition(self.points[i], self.addition(self.points[j], self.points[k]))
					Q = self.addition(self.addition(self.points[i], self.points[j]), self.points[k])
					if P != Q:
						return False
		return True

	# Hàm tính y
	def find_Y(self, X):
		return self.reduceModp((X*X*X + self.a * X + self.b))

	# Hàm tính số lượng điểm của đường cong
	def numberPoints(self):
		return len(self.points)

	# Hàm kiểm tra điểm kì dị
	def discriminant(self):
		D = -16 *(4 * self.a * self.a * self.a + 27 * self.b * self.b)
		return self.reduceModp(D)

	# Hàm lấy mode của một giá trị x trong module p của đường cong elliptic
	def reduceModp(self, x):
		return x % self.p

	# Hàm so sánh hai giá trị trong module p của đường cong elliptic
	def equalModp(self, x, y):
		return self.reduceModp(x - y) == 0




In [279]:
ec = EllipticCurve(a=1, b=1, p= 23)
G = (0, 1)

In [280]:
current_point = G
count = 1
for i in range(30):
    current_point = ec.addition(current_point, G)
    count+=1
    print(f"{count} x G = {current_point} ")
    if (current_point == None):
        break

2 x G = (6, 19) 
3 x G = (3, 13) 
4 x G = (13, 16) 
5 x G = (18, 3) 
6 x G = (7, 11) 
7 x G = (11, 3) 
8 x G = (5, 19) 
9 x G = (19, 18) 
10 x G = (12, 4) 
11 x G = (1, 16) 
12 x G = (17, 20) 
13 x G = (9, 16) 
14 x G = (4, 0) 
15 x G = (9, 7) 
16 x G = (17, 3) 
17 x G = (1, 7) 
18 x G = (12, 19) 
19 x G = (19, 5) 
20 x G = (5, 4) 
21 x G = (11, 20) 
22 x G = (7, 12) 
23 x G = (18, 20) 
24 x G = (13, 7) 
25 x G = (3, 10) 
26 x G = (6, 4) 
27 x G = (0, 22) 
28 x G = None 


In [281]:
cp = ec.multiplication(G, 5)
print(cp)

(18, 3)


# Trao đổi khóa


In [282]:
# Secp256k1: y^2 = x^3 + ax + b = y^2 = x^3 + 7
a = 0; b = 7

# Điểm cơ sở
G = (55066263022277343669578718895168534326250603453777594175500187360389116729240,
     32670510020758816978083085130507043184471273380659243275938904335757337482424)

# modulo
p = pow(2, 256) - pow(2, 32) - pow(2, 9) - pow(2, 8) - pow(2, 7) - pow(2, 6) - pow(2, 4) - pow(2, 0)

# Thứ tự của G trong đường cong
n = 115792089237316195423570985008687907852837564279074904382605163141518161494337
EC = EllipticCurve(a, b, p)

## Trao đổi khóa chia sẻ bí mật ECMQV

In [283]:
import random
# Khóa bí mật tĩnh của Alice
a = random.getrandbits(256)

# Khóa công khai tĩnh của Alice
aG = EC.multiplication(G, a)
Alice = (a, aG)

# Khóa bí mật của tĩnh Bob
c = random.getrandbits(256)
# Khóa công khai của tĩnh Bob
cG = EC.multiplication(G, c)
Bob = (c, cG)

In [284]:
# Chọn ngẫu nhiên b (private key)
b = random.randint(1, n-1)
# Tính public key bG = b × 𝐺
bG = EC.multiplication(G, b)

# Chọn ngẫu nhiên 𝑑 (private key)
d = random.randint(1, n/2-1)
# Tính public key dG = 𝑑 × 𝐺
dG = EC.multiplication(G, d)

In [285]:
import math
def ECMQV(a, b, aG, bG, cG, dG, k):

    n = int(math.log2(k)/2)
    u = bG[0] % (2 ** n) + 2 **(n)
    s = (b + (u*a))
    v = dG[0] % (2 ** n) + 2 **(n)
    Q = EC.multiplication(EC.addition(dG,EC.multiplication(cG,v)),s)
    return Q

## Elliptic curve Deffie-Hellman

In [286]:
import random
# Khóa bí mật của Alice
a = random.getrandbits(256)

# Khóa công khai của Alice
Pa = EC.multiplication(G, a)
Alice = (a, Pa)
# Khóa bí mật của Bob
b = random.getrandbits(256)

# Khóa công khai của Bob
Pb = EC.multiplication(G, b)
Bob = (b, Pb)

In [287]:
Sa = EC.multiplication(Pb,a)
Sb = EC.multiplication( Pa, b)

if Sa == Sb:
    print('Exactly')
else:
    print('That wrong')

Exactly


Như vậy Alice và Bob đã có khóa chung

# Mã hóa và giải mã

## Mã hóa ECIES (The Elliptic Curve Integrated Encryption System)

In [288]:
import hashlib  # Import hashlib for SHA-256 hashing
import hmac  # Import HMAC for generating MAC

# Hàm dẫn xuất khóa từ điểm T
def derive_keys(T):
    tx, ty = T  # Lấy tọa độ x và y của điểm T

    tx_binary = bin(tx)[2:]  # Chuyển đổi tọa độ x thành chuỗi nhị phân
    tx_binary_cropped = tx_binary[0:192]  # Lấy 192 bit đầu tiên của chuỗi nhị phân

    tx_restored = str(int(tx_binary_cropped, 2))  # Chuyển đổi chuỗi nhị phân thành số nguyên

    hashed_tx = bin(int(hashlib.sha256(str.encode(tx_restored)).hexdigest(), 16))[2:]  # Băm giá trị đã phục hồi

    assert len(hashed_tx) == 256  # Kiểm tra chiều dài của giá trị băm

    # Chia giá trị băm thành hai phần để tạo khóa k1 và k2
    k1 = int(hashed_tx[0:128], 2).to_bytes(16, byteorder='big')
    k2 = int(hashed_tx[128:], 2).to_bytes(16, byteorder='big')

    return k1, k2  # Trả về hai khóa k1 và k2

# Hàm tính giá trị MAC từ tin nhắn và khóa
def find_mac(message, key):
    return hmac.new(
        key,
        message,
        hashlib.sha256
    ).hexdigest()  # Trả về giá trị MAC dưới dạng chuỗi hex


In [289]:
# !pip install pycryptodome


In [290]:
from Crypto.Cipher import AES  # Import AES encryption algorithm from Crypto library
from Crypto.Util.Padding import pad, unpad  # Import padding utilities from Crypto library

# Hàm mã hóa sử dụng ECIES
def encrypt_ECIES(m, Sender, Public_key_reciver, EC):
    a, Pa = Sender
    U = Pa  # Tạo điểm U từ khóa công khai của người gửi
    T = EC.multiplication(Public_key_reciver, a)  # Tính điểm T là khóa chung của hai người

    # Bước 2: Sử dụng KDF để dẫn xuất khóa mã hóa đối xứng và khóa MAC
    k1, k2 = derive_keys(T)

    # Bước 3: Mã hóa tin nhắn
    obj_message = AES.new(k1, AES.MODE_CBC, iv=b'\x00' * 16)  # Khởi tạo với một IV toàn số 0
    padded_message = pad(m, AES.block_size)  # Thêm padding để có độ dài là bội số của kích thước block
    c = obj_message.encrypt(padded_message)  # Mã hóa tin nhắn

    # Bước 4: Tính giá trị MAC của tin nhắn đã mã hóa và k2
    r = find_mac(c, k2)

    # Bước 5: Trả về U||c||r
    return U, c, r

# Hàm giải mã sử dụng ECIES
def decrypt_ECIES(M, receiver, EC):
    U, c, r = M
    b, Pb = receiver
    T_prime = EC.multiplication(U, b)  # Tính T' từ khóa công khai của người nhận và điểm U
    k1, k2 = derive_keys(T_prime)  # Dẫn xuất khóa từ điểm chung T'
    r_prime = find_mac(c, k2)  # Tính giá trị MAC từ tin nhắn đã mã hóa và khóa k2

    # Kiểm tra xem giá trị MAC đã tính có khớp với giá trị MAC trong tin nhắn không
    if r_prime == r:
        obj_message = AES.new(k1, AES.MODE_CBC, iv=b'\x00' * 16)  # Khởi tạo với một IV toàn số 0
        decrypted_message = obj_message.decrypt(c)  # Giải mã tin nhắn
        unpadded_message = unpad(decrypted_message, AES.block_size)  # Loại bỏ padding
        return unpadded_message  # Trả về tin nhắn đã giải mã
    else:
        return "Message got changed"  # Trả về thông báo nếu giá trị MAC không khớp

In [291]:
m = b'Hello world!'
M = encrypt_ECIES(m, Alice, Pb, EC)
print('Thông điệp đã mã hóa:',M)
m_decrypted = decrypt_ECIES(M, Bob, EC)
print('Thông điệp sau khi giải mã: ',m_decrypted)

Thông điệp đã mã hóa: ((9199382662290956188446973689789366600841559497838956705733146752256290380445, 85539516375754874138050727133692318087715512807373868305134844599184539453521), b'\x9f\xbeu=;[\x93qk\xfb\xbb]W\x06\xf3}', 'c2f38e3fd51082fd3f80959ce6c6d4241906e4a23213b9a9ba6dd645b19ccbc0')
Thông điệp sau khi giải mã:  b'Hello world!'


## Mã hóa ElGamal

In [292]:
# Hàm chuyển đổi một tin nhắn thành số
def textToInt(text):
    encoded_text = text.encode('utf-8') # Chuyển đổi chuỗi ký tự thành chuỗi bytes bằng cách mã hóa UTF-8
    hex_text = encoded_text.hex() # Chuyển đổi chuỗi bytes thành chuỗi hex
    int_text = int(hex_text, 16) # Chuyển đổi chuỗi hex thành số nguyên, sử dụng hệ cơ số 16
    return int_text

# Hàm chuyển đổi số thành chữ ban đầu
def IntToText(int_text):
    result = ""
    while int_text > 0:
        char_code = int_text % 256  # Lấy phần dư để lấy mã ASCII của ký tự
        result = chr(char_code) + result  # Chuyển mã ASCII thành ký tự và nối vào đầu chuỗi kết quả
        int_text //= 256  # Chia cho 256 để lấy phần nguyên cho lần lặp tiếp theo
    return result

# Hàm chuyển một số thành 1 điểm trên đường cong Elliptic
def IntToPoint(int_text, EC):
    y = EC.find_Y(int_text)
    return (int_text, y) #Dùng hàm find_Y của đường cong EC tương ứng

Bob muốn gửi tin nhắn cho Alice, Bob sẽ tính S dựa vào khóa riêng tư của mình và khóa công khai của Alice
\begin{equation}
S = M + b*P_a
\end{equation}
Và sau đó gửi (Pb, S) cho Alice


In [293]:
# Hàm mã hóa ElGamal
def encrypt_ElGamal(message, sender, Pb, EC):
    '''
    message: thông điệp muốn mã hóa
    sender: là một tuple chưa cặp khóa riêng tư, khóa công khai
    Pb: khóa công khai của người nhận
    EC: đường cong elliptic
    '''
    m = textToInt(message)
    M = IntToPoint(m, EC)
    a, Pa = sender
    S = EC.addition(M, EC.multiplication(Pb, a))
    m = (Pa, S)
    return m

Khi Alice nhận được điểm đã được mã hóa sẽ tiền hành giải mã để nhận thông điệp:
\begin{equation}
M = S - a*P_b
\end{equation}
trong đó Pb, S là thông tin nhận được từ Bob, a là khóa riêng tư của Alice

In [294]:
# Hàm giải mã ElGamal
def decrypt_ElGamal(m, receiver, EC):
    '''
    m: là một điểm đã được mã hóa
    receiver: là một tuple chưa cặp khóa riêng tư, khóa công khai
    EC: đường cong elliptic
    '''
    Pb, S = m
    Pb = Pb[0], -Pb[1]
    M = EC.addition(S, EC.multiplication(Pb, receiver[0]))
    message = IntToText(M[0])
    return message

In [295]:
message = 'Who are u?'
print('Thông điệp muốn gửi: ', message)
print('----------------')
encrypt_M = encrypt_ElGamal(message, sender = Alice, Pb = Bob[1], EC = EC)
hack = IntToText(encrypt_M[1][0])
print('Hacker đọc lén tin nhắn: ')
print(hack)
print('----------------')
decrypt_M = decrypt_ElGamal(encrypt_M, receiver= Bob,  EC = EC)
print('Thông điệp sau khi giải mã: ', decrypt_M)

Thông điệp muốn gửi:  Who are u?
----------------
Hacker đọc lén tin nhắn: 
ú /±%T÷>ãN )1¼¿Qù'ÐxÝt¢W
----------------
Thông điệp sau khi giải mã:  Who are u?


# Chữ ký số

## Chữ ký số ElGamal

In [296]:
import random
# Hàm sinh chữ ký số ELGamal
def generation_DS_ElGamal(x, m, EC, G, n):
    '''
    x: khóa riêng tư của người gửi
    m:  thông điệp muốn gửi
    EC: đường cong elliptic
    G: điểm cở sở đã chọn
    n: thứ tự điểm cơ sở
    '''
    k = random.randint(1, n-2)  # Chọn một số ngẫu nhiên k trong khoảng [1, n-1]
    R = EC.multiplication(G, k)  # Nhân G với k để có điểm ngẫu nhiên R
    # Tính s bằng cách sử dụng modular inverse và modular reduction
    k_inv = pow(k, -1, n)  # Modular inverse của k (mod n)
    s = (k_inv * (m - x * R[0])) % n # Modular reduction sau khi nhân

    return R, s  # Chữ ký là cặp (R, s)

# Hàm xác thực chữ ký số ELGamal
def verify_DS_ElGamal(Y, m, DS, EC, G):
    '''
    Y: khóa công khai của người gửi
    m:  thông điệp nhận được
    DS: chữ ký nhận được
    EC: đường cong elliptic
    G: điểm cở sở đã chọn
    '''
    R, s = DS  # Unpack the signature
    V1 = EC.addition(EC.multiplication(Y, R[0]), EC.multiplication(R, s))
    V2 = EC.multiplication(G, m)
    return V1 == V2


In [297]:
message = 'Hello world!'
m = textToInt(message)
DS = generation_DS_ElGamal(a, m, EC, G, n)
print('Chữ ký: ', DS)
# Bob receives the Digital Signature and checks it.
message1 = 'Hello'
m1 = textToInt(message1)
if verify_DS_ElGamal(Pa, m, DS, EC, G):
    print("Chữ ký được xác thực!")
else:
    print("Chữ ký không tồn tại!")

Chữ ký:  ((34249792283179824075781643903209133766320263553267269695164239267854897269216, 106181456649484741061102738090276749346933843425861126669785285923548984436671), 90280146605046852321696330082356034949617002261714859871878014598064410945763)
Chữ ký được xác thực!


## ECDSA - Elliptic curve digital signature algorithm

In [298]:
# ECDSA khởi tạo khóa
def generate_key_pair():
    # Chọn ngẫu nhiên 𝑑 (private key) trong khoảng [1, n-1]
    private_key = random.randint(1, n-1)
    # Tính public key 𝑄 = 𝑑 × 𝐺
    public_key = EC.multiplication(G, private_key)
    return private_key, public_key

In [299]:
import hashlib
 # ECDSA sinh chữ ký số
def sign_ECDSA(message, private_key):

    k = random.randint(1, n-1) # 1. Chọn số k ngẫu nhiên thuộc [1, n-1],
    x, _ = EC.multiplication(G, k) # 2. (x1, y1) = kG
    r = x % n # 3. r = x1 mod n

    if r == 0:
        return sign_ECDSA(message, private_key)  # 4. r = 0 quay lại bước 1
    message = message.encode()
    e = int.from_bytes(hashlib.sha256(message).digest(), byteorder='big') % n  # 5. e = H(m) chuyển đổi tin nhắn thành 1 số bằng hàm băm
    s = (pow(k, -1, n) * (e + private_key * r)) % n    # 6. s = k^-1 * (e+ d*r)

    if s == 0:
        return sign_ECDSA(message, private_key) # 7. s = 0 quay lại bước 1
    return r, s

# ECDSA xác thực chữ ký số
def verify_ECDSA(message, signature, public_key):
    message = message.encode()
    r, s = signature
    if not (0 < r < n and 0 < s < n):
        return False
    e = int.from_bytes(hashlib.sha256(message).digest(), byteorder='big') % n
    w = pow(s, -1, n)
    u1 = (e * w) % n
    u2 = (r * w) % n
    x, _ = EC.addition(EC.multiplication(G, u1), EC.multiplication(public_key, u2))
    if (r % n) == (x % n):
        return True
    else:
        return False

In [300]:
message = "Hello, world!"
private_key, public_key = generate_key_pair()
print("Khóa bí mật:", hex(private_key))
print("Khóa công khai:", public_key)

signature = sign_ECDSA(message, private_key)
print("Chữ ký số:", signature)



Khóa bí mật: 0xa736d2e6c6a4f3e813e000dd43fec986bbed5b0f5ca432a37330cd39fff620bf
Khóa công khai: (38869028779151858866788440335097450758970308129976736509059719121874116110610, 58135296651230529768188213718840266786906436647239186244783461198979582152956)
Chữ ký số: (64679407910741613320601293418527975451003818173667401524163551345121264056055, 66422986853658927573547579949287368733665125606744107048436397648382738773698)


In [301]:
if verify_ECDSA(message, signature, public_key):
    print("Chữ ký được xác thực!")
else:
    print("Chữ ký không tồn tại")

Chữ ký được xác thực!


# Demo

In [302]:
import random
class Chat:
    def __init__(self, a, b, p, G, n, id1, id2):
        self.a = a
        self.b = b
        self.p = p
        self.G = G
        self.n = n
        self.id1 = id1
        self.id2 = id2

        self.EC = EllipticCurve(self.a, self.b, self.p)

        self.private_key_id1 = random.getrandbits(256)
        self.public_key_id1 = self.EC.multiplication(self.G, self.private_key_id1)
        self.member1 = (self.private_key_id1, self.public_key_id1)

        self.private_key_id2 = random.getrandbits(256)
        self.public_key_id2 = self.EC.multiplication(self.G, self.private_key_id2)
        self.member2 = (self.private_key_id2, self.public_key_id2)

    def send_message(self, sender, receiver, message):
        # Mã hóa tin nhắn bằng ElGamal
        message_encrypt = encrypt_ElGamal(message, sender, receiver[1], self.EC)
        # Sinh chữ ký ứng với thông điệp đã mã hóa
        signature = generation_DS_ElGamal(sender[0], message_encrypt[1][0], self.EC, self.G, self.n)
        return message_encrypt, signature

    def receive_message(self, sender, receiver, M):
        message_encrypt, signature = M
        verify = verify_DS_ElGamal(sender[1], message_encrypt[1][0], signature, self.EC, self.G)
        if verify:
            message_original = decrypt_ElGamal(message_encrypt, receiver, self.EC)
            return message_original
        else:
            return "Tin nhắn đã bị hacker thay đổi"

In [303]:
# Secp256k1: y^2 = x^3 + ax + b = y^2 = x^3 + 7
a = 0; b = 7

# Điểm cơ sở
G = (55066263022277343669578718895168534326250603453777594175500187360389116729240,
     32670510020758816978083085130507043184471273380659243275938904335757337482424)

# modulo
p = pow(2, 256) - pow(2, 32) - pow(2, 9) - pow(2, 8) - pow(2, 7) - pow(2, 6) - pow(2, 4) - pow(2, 0)

# Thứ tự của G trong đường cong
n = 115792089237316195423570985008687907852837564279074904382605163141518161494337
EC = EllipticCurve(a, b, p)

In [304]:
id1 = 1
id2 = 2
chat_1_2 = Chat(a, b, p, G, n, id1, id2)

In [305]:
message1 = 'Help me guy!'
sent_message = chat_1_2.send_message(chat_1_2.member1, chat_1_2.member2, message1)
print('Tin nhắn sẽ được gửi đi: ', sent_message)

Tin nhắn sẽ được gửi đi:  (((6509315782644017946050044136042187091860350590192491131051034314628072580437, 90959383961144886320975650813931367826135868194158498877373265869530202937390), (61600008715244206007339365931951334218512794454086922342774975061747878252476, 69074669053992011001258560210164408167686878589693012144009531367227246101399)), ((1875872991311864312082179152451404023617884040764921193790108470186169101861, 47499100602540046046255092763970267123584548418191104486110586256425057251997), 14316541502373310342952684843660436180093370189210916261411160273728534295573))


In [306]:
hack = IntToText(sent_message[0][1][0])
print("Tin nhắn hacker đọc được: ", hack)

Tin nhắn hacker đọc được:  0^þÑWìkz-úÜxhzî tÃ*=¦¶¿(rÏ¼


In [307]:
message_received = chat_1_2.receive_message(chat_1_2.member1, chat_1_2.member2, sent_message)
print("Tin nhắn nhận được sau khi giải mã: ", message_received)

Tin nhắn nhận được sau khi giải mã:  Help me guy!


In [308]:
class banking:
  def __init__(self, a, b, p, G, n, id1, id2):
      self.a = a
      self.b = b
      self.p = p
      self.G = G
      self.n = n
      self.id1 = id1
      self.id2 = id2

      self.EC = EllipticCurve(self.a, self.b, self.p)

      self.private_key_id1 = random.getrandbits(256)
      self.public_key_id1 = self.EC.multiplication(self.G, self.private_key_id1)
      self.member1 = (self.private_key_id1, self.public_key_id1)

      self.private_key_id2 = random.getrandbits(256)
      self.public_key_id2 = self.EC.multiplication(self.G, self.private_key_id2)
      self.member2 = (self.private_key_id2, self.public_key_id2)

  def transfer_money(self, sender, reciver, transfer_amount):
      singature = sign_ECDSA(transfer_amount, sender[0])
      return singature

  def complete_transfer(self, sender, reciver, transfer_amount, singature):
      verify = verify_ECDSA(transfer_amount, singature, sender[1])
      if verify:
          transfer_amount = int(transfer_amount)
          if (sender[0] == self.member1[0]):
            self.id1[1], self.id2[1] = self.id1[1] - transfer_amount, self.id2[1] + transfer_amount
          if (sender[0] == self.member2[0]):
            self.id2[1], self.id1[1] = self.id2[1] - transfer_amount, self.id1[1] + transfer_amount
          return 'Giao dịch thành công!'
      return "Giao dịch thất bại!"



In [309]:
id3 = [3, 500]
id4 = [4, 2000]
baking34 = banking(a, b, p, G, n, id3, id4)


In [310]:
transfer_amount = '100'
if(int(transfer_amount)> baking34.id1[1]): #Kiểm tra số tiền của người 1 muốn gửi
  print('Số dư không đủ')
else:
  sig = baking34.transfer_money(baking34.member1, baking34.member2, transfer_amount) #Sinh chữ ký cho giao dịch giao dịch
  print(sig)
  print('ID và số tiền người 1', baking34.id1)
  print('ID và số tiền người 2', baking34.id2)


(23256255436789568902178737609228102288699521171331635094202278219165157183652, 14603822963266198580461745505822131895897254579016058946669473441364807734901)
ID và số tiền người 1 [3, 500]
ID và số tiền người 2 [4, 2000]


In [311]:
transfer_amount = '100'
result = baking34.complete_transfer(baking34.member1, baking34.member2, transfer_amount, sig) #Hoàn tất hoặc hủy giao dịch
print(result)
print('ID và số tiền người 1', baking34.id1)
print('ID và số tiền người 2', baking34.id2)

Giao dịch thành công!
ID và số tiền người 1 [3, 400]
ID và số tiền người 2 [4, 2100]
